<a href = "https://colab.research.google.com/github/flyingzaptop/pyton-colab/blob/main/pipelines%20%D0%94%D0%D0%BC%BC%B0%D1BD1BD Target =" _ Parent "> <mmg src = „https://colab.research.google.com/assets/colab-badge.svg” alt = "Otwórz w colab"/> </a>

In [249]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [250]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/House_Rent_Dataset.csv", index_col="Unnamed: 0")

[Informacje o danych] (https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset)

In [251]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100.0,Ground out of 2,NaN,NaN,Kolkata,Unfurnished,Bachelors/Family,2.0,Contact Owner
1,2022-05-13,2.0,20000.0,800.0,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1.0,NaN
2,2022-05-16,2.0,17000.0,1000.0,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1.0,Contact Owner
3,2022-07-04,2.0,10000.0,800.0,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1.0,Contact Owner
4,NaN,2.0,7500.0,850.0,1 out of 2,Carpet Area,NaN,Kolkata,Unfurnished,Bachelors,1.0,Contact Owner


In [252]:
df = df[['Size', 'Furnishing Status', 'City', 'Rent']]
df.head()

,Size,Furnishing Status,City,Rent
0,1100.0,Unfurnished,Kolkata,10000.0
1,800.0,Semi-Furnished,Kolkata,20000.0
2,1000.0,Semi-Furnished,Kolkata,17000.0
3,800.0,Unfurnished,Kolkata,10000.0
4,850.0,Unfurnished,Kolkata,7500.0


# Zadanie 1
Podaj główne cechy danych liczbowych, podziel na szkolenia i czaty testowe. Ukierunkowany znak `Rent '

In [253]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [254]:
from types import SimpleNamespace
y = df['Rent']
X = df.drop('Rent', axis=1)
X

,Size,Furnishing Status,City
0,1100.0,Unfurnished,Kolkata
1,800.0,Semi-Furnished,Kolkata
2,1000.0,Semi-Furnished,Kolkata
3,800.0,Unfurnished,Kolkata
4,850.0,Unfurnished,Kolkata
...,...,...,...
4741,1000.0,Semi-Furnished,Hyderabad
4742,2000.0,Semi-Furnished,Hyderabad
4743,1750.0,Semi-Furnished,Hyderabad
4744,1500.0,Semi-Furnished,Hyderabad


In [255]:
y.isna().sum()

377

In [256]:
nums = X.select_dtypes(include="number").columns
cats = X.select_dtypes(include="object").columns

In [257]:
imputer = SimpleImputer(strategy='median')
y.fillna(y.median(), inplace=True)
y.isna().sum()

0

In [258]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42)

# Zadanie 2
Utwórz rurociąg do przetwarzania kolumn numerycznych (wypełnienie luk)

In [259]:
num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
])

# Zadanie 3
Utwórz rurociąg, aby przetwarzać kategoryczne kolumny (wypełnianie luk i kodowanie)

In [260]:
cat_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output = False)),
])

# Zadanie 4
Połącz poprzedni rurociąg z jednym kolumnowym transformerem

In [261]:
preproc = ColumnTransformer(
    transformers = [
      ('num', num_pipeline, nums),
      ('cat', cat_pipeline, cats),
])
preproc.set_output(transform="pandas")

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['Size'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(sparse_output=False))]),
                                 Index(['Furnishing Status', 'City'], dtype='object'))])

In [262]:
preproc.fit(X_train)
X_prep = preproc.transform(X_train)
X_prep.isna().sum()

,0
num__Size,0
cat__Furnishing Status_Furnished,0
cat__Furnishing Status_Semi-Furnished,0
cat__Furnishing Status_Unfurnished,0
cat__City_Bangalore,0
cat__City_Chennai,0
cat__City_Delhi,0
cat__City_Hyderabad,0
cat__City_Kolkata,0
cat__City_Mumbai,0


# Zadanie 5
Utwórz ostateczny model i przeszkolić go

In [263]:
model = Pipeline([
    ('preproc', preproc),
    ('model', LinearRegression())
])


In [264]:
model

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])

In [265]:
model.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])

In [266]:
model.score(X_test,y_test)

0.41889837000424146

# Zadanie 6
Wydaj podstawowe wskaźniki modelu

In [269]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R-squared: {r2}")


MSE: 2235661990.969119
R-squared: 0.41889837000424146


# Zadanie 7
Zapisz model, pobierz go i poprzedzaj cenę samochodu

In [270]:
import pickle

with open('house_rent_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('house_rent_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)



In [271]:
data = pd.DataFrame({'Size': [900],
                     'Furnishing Status': ['Semi-Furnished'],
                     'City': ['Mumbai']})

data

,Size,Furnishing Status,City
0,900,Semi-Furnished,Mumbai


In [272]:
predicted_price = loaded_model.predict(data)
print(f"Прогнозована ціна оренди: {predicted_price[0]}")

Прогнозована ціна оренди: 62595.47444581421
